# World Bank Indicators API Example

> The following is an example of a [Jupyter notebook](https://jupyter.org) - a tutorial of how to retrieve data from the [World Bank Indicators API](https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-about-the-indicators-api-documentation) - that illustrates how to use computational content with the [template](https://worldbank.github.io/template). 

## Requirements

In [1]:
import itertools

import pandas
import requests
from bokeh.palettes import Spectral6
from bokeh.plotting import figure, output_notebook, show

## Data Retrieval

In this example, we retrieve **Population, total** (`SP.POP.TOTL`) from the [World Bank Indicators](https://data.worldbank.org/indicator) for [BRICS](https://infobrics.org).

In [2]:
url = "https://api.worldbank.org/v2/country/chn;bra;ind;rus;zaf/indicator/SP.POP.TOTL?format=json&per_page=10000"

Let's use [requests](https://requests.readthedocs.io) to send a GET request,

In [3]:
r = requests.get(url)

Now, let's normalize and create `pandas.DataFrame` from the response,

In [4]:
# normalize
data = pandas.json_normalize(r.json()[-1])

# create dataframe
df = pandas.DataFrame.from_dict(data)

```{tip}
Alternatively, the World Bank API supports downloading the data as an [archive](http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL?date=2000&source=2&downloadformat=csv). 
```

Let's take a look at the dataframe, 

In [5]:
df = df.pivot_table(values="value", index="date", columns="countryiso3code")
df = df / 1e6  # scaling
df

countryiso3code,BRA,CHN,IND,RUS,ZAF
date,,,,,
1960,73.092515,667.070,445.954579,119.897000,16.520441
1961,75.330008,660.330,456.351876,121.236000,16.989464
1962,77.599218,665.770,467.024193,122.591000,17.503133
1963,79.915555,682.335,477.933619,123.960000,18.042215
1964,82.262794,698.355,489.059309,125.345000,18.603097
...,...,...,...,...,...
2018,210.166592,1402.760,1369.003306,144.477859,57.339635
2019,211.782878,1407.745,1383.112050,144.406261,58.087055
2020,213.196304,1411.100,1396.387127,144.073139,58.801927


## Visualization

Let's now plot the data as a time series using [Bokeh](https://docs.bokeh.org).

In [6]:
output_notebook()

p = figure(title="Population, total (World Bank)", width=700, height=600)

# colors
colors = itertools.cycle(Spectral6)

# plotting the line graph
for column, color in zip(df.columns, colors):
    p.line(
        df.index,
        df[column],
        legend_label=column,
        color=color,
        line_width=2,
    )

p.legend.location = "right"
p.legend.click_policy = "mute"
p.title.text_font_size = "12pt"

p.xaxis.axis_label = "Year"
p.yaxis.axis_label = "Population, total (in millions)"

show(p)

Loading BokehJS ...